In [ ]:
def slab_source(Nx,Sig_s,Sig_a,thickness,a,b,N,Q,
                implicit_capture = True, cutoff = 1.0e-3, stratified = [1,1]):
    """Compute the fraction of neutrons that leak through a slab
    Inputs:
    Nx: The number of grid points
    Sig_s: The scattering macroscopic x-section
    Sig_a: The absorption macroscopic x-section
    thickness: Width of the slab
    a,b: Endpoints of Source
    N: Number of neutrons to simulate
    implicit_capture: Do we run implicit capture
    cutoff: At what level do we stop implicit capture
    stratified: Use stratified sampling in space and angle
                Specify a list of length two with the number of
                strata in each dimension; default [1,1] for unstratified
    Returns:
    transmission: The fraction of neutrons that made it through
    scalar_flux: The scalar flux in each of the Nx cells
    scalar_flux_tl: The scalar flux in each of the Nx cells
                    from track length estimator
    X: The value of the cell centers in the mesh
    """
    imp_input = implicit_capture
    dx = thickness/Nx
    X = np.linspace(dx*0.5, thickness - 0.5*dx,Nx)
    scalar_flux = np.zeros(Nx)
    scalar_flux_tl = np.zeros(Nx)
    assert (Sig_s.size == Nx) and (Sig_a.size == Nx)
    Sig_t = Sig_a + Sig_s
    iSig_t = 1.0/Sig_t
    iSig_s = 1.0/(Sig_s+1.0e-14)
    iSig_a = 1.0/(Sig_a+1.0e-14)
    leak_left = 0.0
    leak_right = 0
    N = int(N)
    #make a vector of the initial positions and mus
    samples = strat_sample_2D(N,stratified[0],stratified[1])
    xs = samples[:,0]*(b-a) + a #adjust to bounds of source
    mus = (samples[:,1]-0.5)*2 #shift to range -1 to 1
    N = int(xs.size)
    #the initial weight does not change
    init_weight = Q*thickness/N
    for i in range(N):
        mu = mus[i]
    x = xs[i]
    alive = 1
    weight = init_weight
    #which cell am I in
    cell = int(x/dx)
    implicit_capture = imp_input
    while (alive):
        if (weight < cutoff*init_weight):
            implicit_capture = False
        if (implicit_capture):
            l = -math.log(1-random.random())*iSig_s[cell]
        else:
            #get distance to collision
            l = -math.log(1-random.random())*iSig_t[cell]
        #compare distance to collision to distance to cell edge
        distance_to_edge = ((mu > 0.0)*( (cell+1)*dx - x) +
                            (mu<0.0)*( x - cell*dx) + 1.0e-8)/math.fabs(mu)
        if (distance_to_edge < l):
            l = distance_to_edge
            collide = 0
        else:
            collide = 1
        x += l*mu #move particle
        #score track length tally
        if (implicit_capture):
            scalar_flux_tl[cell] += weight*(1.0 -
                                            math.exp(-l*Sig_a[cell]))*iSig_a[cell]
        else:
            scalar_flux_tl[cell] += weight*l
        if (implicit_capture):
            weight *= math.exp(-l*Sig_a[cell])
        #still in the slab?
        if (math.fabs(x-thickness) < 1.0e-14) or (x > thickness):
            leak_right += weight
            alive = 0
        elif (x<= 1.0e-14):
            alive = 0
            leak_left += weight
        else:
            cell= int(x/dx) #compute cell particle collision is in
            if (implicit_capture):
                if (collide):
                    mu = random.uniform(-1,1)
                scalar_flux[cell] += weight*iSig_s[cell]/dx
            else: #scatter or absorb
                scalar_flux[cell] += weight*iSig_t[cell]/dx
                if (collide) and (random.random() < Sig_s[cell]*iSig_t[cell]):
                    #scatter, pick new mu
                    mu = random.uniform(-1,1)
                elif (collide): #absorbed
                    alive = 0
    return leak_left,leak_right, scalar_flux, scalar_flux_tl/dx, X, N